In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("./drive/MyDrive/")

In [3]:
!pip install transformers

     |████████████████████████████████| 2.5MB 7.2MB/s 
     |████████████████████████████████| 901kB 50.2MB/s 
     |████████████████████████████████| 3.3MB 32.5MB/s 


In [4]:
import pandas as pd
df = pd.read_csv('data1.csv')
df.head()

,Title,Conference
0,Pre-pandemic mental and physical health as pre...,medRxiv : the preprint server for health sciences
1,Occupational Characteristics and Management Me...,Frontiers in public health
2,Perceived Social Norms as Determinants of Adhe...,Frontiers in public health
3,Electrolyzed Water Generated On-Site as a Prom...,Frontiers in public health
4,Utilization of Inpatient Mental Health Care in...,Frontiers in public health


In [5]:
df['Conference'].value_counts()

BMJ (Clinical research ed.)                                                                                                        1624
International journal of environmental research and public health                                                                  1470
PloS one                                                                                                                           1338
Journal of medical virology                                                                                                         994
medRxiv : the preprint server for health sciences                                                                                   917
Cureus                                                                                                                              786
International journal of infectious diseases : IJID : official publication of the International Society for Infectious Diseases     704
Lancet (London, England)                        

In [6]:
import torch
from tqdm.notebook import tqdm
import pandas as pd
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

possible_labels = df.Conference.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'BMJ (Clinical research ed.)': 9,
 'Clinical infectious diseases : an official publication of the Infectious Diseases Society of America': 17,
 'Cureus': 19,
 'Frontiers in psychology': 2,
 'Frontiers in public health': 1,
 'International journal of environmental research and public health': 4,
 'International journal of infectious diseases : IJID : official publication of the International Society for Infectious Diseases': 5,
 'JAMA': 12,
 'Journal of medical Internet research': 13,
 'Journal of medical virology': 10,
 'Lancet (London, England)': 7,
 'Nature': 16,
 'PloS one': 11,
 'Science (New York, N.Y.)': 14,
 'Scientific reports': 8,
 'The Journal of infection': 6,
 'The New England journal of medicine': 18,
 'The Science of the total environment': 3,
 'bioRxiv : the preprint server for biology': 15,
 'medRxiv : the preprint server for health sciences': 0}

In [7]:
df['label'] = df.Conference.replace(label_dict)

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['Conference', 'label', 'data_type']).count()

Title
Conference                                         label data_type       
BMJ (Clinical research ed.)                        9     train       1380
                                                         val          244
Clinical infectious diseases : an official publ... 17    train        527
                                                         val           93
Cureus                                             19    train        668
                                                         val          118
Frontiers in psychology                            2     train        516
                                                         val           91
Frontiers in public health                         1     train        462
                                                         val           82
International journal of environmental research... 4     train       1250
                                                         val          220
International journal of infectious diseases : ... 5     train        598
                                                         val          106
JAMA                                               12    train        488
                                                         val           86
Journal of medical Internet research               13    train        396
                                                         val           70
Journal of medical virology                        10    train        845
                                                         val          149
Lancet (London, England)                           7     train        570
                                                         val          100
Nature                                             16    train        442
                                                         val           78
PloS one                                           11    train       1137
                                                         val          201
Science (New York, N.Y.)                           14    train        370
                                                         val           65
Scientific reports                                 8     train        512
                                                         val           90
The Journal of infection                           6     train        414
                                                         val           73
The New England journal of medicine                18    train        476
                                                         val           84
The Science of the total environment               3     train        364
                                                         val           64
bioRxiv : the preprint server for biology          15    train        469
                                                         val           83
medRxiv : the preprint server for health sciences  0     train        779
                                                         val          138

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].Title.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].Title.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2111: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [10]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [11]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [12]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 10

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)


In [13]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [14]:
import random
import numpy as np

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
device="cuda"

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    model.to(device)
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')



Epoch 1
Training loss: 2.2063994256735864
Validation loss: 1.9176290028097245
F1 Score (Weighted): 0.35070226887048656



Epoch 2
Training loss: 1.6949661427275233
Validation loss: 1.776827440264591
F1 Score (Weighted): 0.4129191146770753



Epoch 3
Training loss: 1.3726970042911568
Validation loss: 1.77840299449186
F1 Score (Weighted): 0.43454796267572726



Epoch 4
Training loss: 1.0865169597406945
Validation loss: 1.9312253464116471
F1 Score (Weighted): 0.4521629447799161



Epoch 5
Training loss: 0.8415265747354699
Validation loss: 2.1599096129252264
F1 Score (Weighted): 0.45944058887836897



Epoch 6
Training loss: 0.655235010080572
Validation loss: 2.4298272402702374
F1 Score (Weighted): 0.46264328130859583



Epoch 7
Training loss: 0.5036809344819855
Validation loss: 2.8033360011776405
F1 Score (Weighted): 0.4610929021320626



Epoch 8
Training loss: 0.3893486369467671
Validation loss: 3.094158829303498
F1 Score (Weighted): 0.4584490120369295



Epoch 9
Training loss: 0.30869501328252213
Validation loss: 3.2735532453916236
F1 Score (Weighted): 0.45748017556677234



Epoch 10
Training loss: 0.259521927600947
Validation loss: 3.3348210646557206
F1 Score (Weighted): 0.4599825550022605

